In [34]:
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
import warnings

import wandb
from otc.metrics.metrics import effective_spread
from scipy.stats import wilcoxon
from tqdm.auto import tqdm


In [35]:
# set here globally
EXCHANGE = "cboe" # "ise"
MODELS = ["gbm", "classical"]  # ["gbm", "classical"]
SUBSET = "test"  # "all"
STRATEGY = "transfer" # "supervised" 


In [36]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}"
DATASET = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(DATASET)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in MODELS:
    results = f"fbv/thesis/{EXCHANGE}_{model}_{STRATEGY}_{SUBSET}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


wandb: Downloading large artifact cboe_transfer_raw:latest, 663.29MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.1
wandb: Downloading large artifact cboe_gbm_transfer_test:latest, 142.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_transfer_test:latest, 653.40MB. 1 files... 
/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
Done. 0:0:12.8


In [37]:
# p. 35-38
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if SUBSET == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif SUBSET == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )


results = []
for i, model in tqdm(enumerate(MODELS)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=MODELS)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:06,  3.01s/it]


In [38]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        # ("gbm", "gbm(classical-retraining)"),
        ("gbm", "gbm(classical-size)"),
        # ("gbm", "gbm(classical-size-retraining)"),
        ("gbm", "gbm(ml)"),
        # ("gbm", "gbm(ml-retraining)"),
        ("gbm", "gbm(semi-classical)"),
        ("gbm",'gbm(semi-classical-size)'),
        ("gbm",'gbm(semi-ml)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "quote(best)->quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [39]:
results_data


gbm                                                  \
         gbm(classical) gbm(classical-size) gbm(ml) gbm(semi-classical)   
index                                                                     
24700819             -1                  -1      -1                  -1   
24700261             -1                  -1       1                  -1   
24700842             -1                  -1      -1                  -1   
24695039              1                   1       1                   1   
24700841              1                   1       1                   1   
...                 ...                 ...     ...                 ...   
37155409              1                   1       1                   1   
37155410              1                   1       1                   1   
37104659             -1                  -1      -1                  -1   
37108365              1                   1       1                   1   
37155411              1                   1       1                   1   

                                               classical            \
         gbm(semi-classical-size) gbm(semi-ml)  tick(ex) quote(ex)   
index                                                                
24700819                       -1           -1         1         0   
24700261                       -1            1         1        -1   
24700842                       -1           -1         1         0   
24695039                        1            1        -1         0   
24700841                        1            1         1         1   
...                           ...          ...       ...       ...   
37155409                        1            1         1        -1   
37155410                        1            1         1         1   
37104659                       -1           -1        -1         1   
37108365                        1            1        -1        -1   
37155411                        1            1        -1        -1   

                                                         \
         quote(best)->quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                     
24700819                     -1     -1      -1       -1   
24700261                     -1     -1      -1       -1   
24700842                     -1      1       1        1   
24695039                      1     -1      -1       -1   
24700841                      1      1       1        1   
...                         ...    ...     ...      ...   
37155409                     -1     -1      -1       -1   
37155410                      1      1       1        1   
37104659                      1      1       1        1   
37108365                     -1     -1      -1       -1   
37155411                     -1     -1      -1       -1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
24700819                                                 -1                             
24700261                                                 -1                             
24700842                                                 -1                             
24695039                                                  1                             
24700841                                                  1                             
...                                                     ...                             
37155409                                                 -1                             
37155410                                                  1                             
37104659                                                 -1                             
37108365                                                  1                             
3715

In [40]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Quote(Ex)->Depth(Best)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all": "All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [41]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)

    res = (
        res.apply(highlight_max, props="font-weight:bold;", axis=bold_axis)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"../reports/{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [42]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


## Unclassified by method

In [43]:
results_data.head()


gbm                                                  \
         gbm(classical) gbm(classical-size) gbm(ml) gbm(semi-classical)   
index                                                                     
24700819             -1                  -1      -1                  -1   
24700261             -1                  -1       1                  -1   
24700842             -1                  -1      -1                  -1   
24695039              1                   1       1                   1   
24700841              1                   1       1                   1   

                                               classical            \
         gbm(semi-classical-size) gbm(semi-ml)  tick(ex) quote(ex)   
index                                                                
24700819                       -1           -1         1         0   
24700261                       -1            1         1        -1   
24700842                       -1           -1         1         0   
24695039                        1            1        -1         0   
24700841                        1            1         1         1   

                                                         \
         quote(best)->quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                     
24700819                     -1     -1      -1       -1   
24700261                     -1     -1      -1       -1   
24700842                     -1      1       1        1   
24695039                      1     -1      -1       -1   
24700841                      1      1       1        1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
24700819                                                 -1                             
24700261                                                 -1                             
24700842                                                 -1                             
24695039                                                  1                             
24700841                                                  1

In [44]:
unclassified = (
    (results_data[results_data == 0.0].count(axis=0) / len(results_data.index))
    .sort_values(ascending=False)
    .to_frame(name="unclassified")
)


In [45]:
unclassified.style.pipe(
    set_tex_style,
    caption=(f"{KEY}-unclassified-long", "{key}-unclassified-short"),
    label=f"{KEY.lower()}-unclassfied",
    bold_axis=0,
)
unclassified


unclassified
classical tick(ex)                                                0.099217
          quote(ex)                                               0.060515
          quote(best)->quote(ex)                                  0.033452
          emo(ex)                                                 0.026380
          clnv(ex)                                                0.020618
          lr(ex)                                                  0.001719
          trade_size(ex)->quote(best)->quote(ex)->depth(b...      0.000040
gbm       gbm(classical)                                          0.000000
          gbm(classical-size)                                     0.000000
          gbm(ml)                                                 0.000000
          gbm(semi-classical)                                     0.000000
          gbm(semi-classical-size)                                0.000000
          gbm(semi-ml)                                            0.000000

## Fill in unclassified

Unclassified are `0`.

In [46]:
rng = np.random.default_rng(42)

# replace 0 -> nan -> [-1,1]
results_data.replace(0, np.nan, inplace=True)
filler = pd.DataFrame(
    rng.choice(a=[-1, 1], size=results_data.shape),
    index=results_data.index,
    columns=results_data.columns,
)
results_data.fillna(filler, inplace=True)


## DistributionⓂ️

In [47]:
X_print["buy_sell"].value_counts() / len(X_print)

-1    0.538722
1     0.461278
Name: buy_sell, dtype: Float64

### Robustness Checks

In [48]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
# half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        (bid < trade_price) & (trade_price < ask),
        1,  # inside
        np.where(
            (trade_price == bid) | (trade_price == ask),
            2,  # at quotes
            np.where((trade_price < bid) | (ask < trade_price), 3, 4),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [49]:
X_print = pd.concat([X_print, results_data], axis=1)


In [50]:
X_print.head().T


index,24700819,24700261,24700842,24695039,24700841
buy_sell,-1,-1,-1,1,-1
TRADE_PRICE,1.73,0.6,1.3,1.28,1.04
OPTION_TYPE,C,C,C,C,C
issue_type,Others,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(0,1]","(1,3]","(1,3]","(0,1]",>11
year_binned,2015,2015,2015,2015,2015
ttm_binned,(1-2],<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.9-1.1],(0.9-1.1],(0.9-1.1]
prox_q_binned,unknown,at quotes,unknown,unknown,at quotes
mid,NaN,0.8,NaN,NaN,1.0


## Accuracy Calculation

In [51]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    # result_df.style.pipe(
    #     set_tex_style,
    #     caption=(f"long-tbd", "short-tbd"),
    #     label=f"{KEY.lower()}-{criterion.lower()}",
    # )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [03:30<00:00, 26.29s/it]


In [52]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [53]:
master


gbm                                 \
                               gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type       C                 65.512657           71.752108  74.326804   
                  P                 66.606767           72.306912  74.451890   
Security Type     Index option      60.309635           67.469911  72.346139   
                  Others            69.039887           74.160293  75.569654   
                  Stock option      65.318447           71.559532  74.093552   
Trade Size        (0,1]             62.838447           70.814533  73.435935   
                  (1,3]             64.907873           71.349116  73.647650   
                  (3,5]             65.530185           72.022665  74.190970   
                  (5,11]            66.584846           71.632683  74.157743   
                  >11               71.162306           74.596742  76.754620   
Year              2015              65.686244           71.517005  74.244782   
                  2016              65.599281           71.689210  74.197954   
                  2017              66.489807           72.401584  74.599385   
Time to Maturity  <= 1              66.871548           72.041871  74.239008   
                  (1-2]             67.586737           72.695265  75.288671   
                  (2-3]             66.036792           72.431363  74.814515   
                  (3-6]             65.029245           72.896311  75.145620   
                  (6-12]            64.199075           71.966191  74.428193   
                  > 12              56.015639           66.891540  71.214760   
Moneyness         <= 0.7            65.757089           73.047039  75.613996   
                  (0.7-0.9]         66.286506           72.245056  74.703549   
                  (0.9-1.1]         66.989236           72.658136  74.765376   
                  (1.1-1.3]         58.149819           66.258707  69.844057   
                  > 1.3             56.912370           64.579940  70.634098   
Location to Quote at mid            61.013630           67.961360  69.818239   
                  inside            68.837832           72.024322  73.714183   
                  at quotes         54.152059           74.856559  81.415458   
                  outside           70.802418           70.149766  70.644408   
                  unknown           83.626840           83.392035  83.997110   
All trades        all               66.010410           72.004510  74.383710   

                                                                             \
                               gbm(semi-classical) gbm(semi-classical-size)   
Option Type       C                      65.659616                71.649785   
                  P                      66.791449                72.207775   
Security Type     Index option           60.648176                67.177757   
                  Others                 69.252296                74.122539   
                  Stock option           65.445187                71.451464   
Trade Size        (0,1]                  62.968438                70.806588   
                  (1,3]                  65.096787                71.347635   
                  (3,5]                  65.715104                72.068491   
                  (5,11]                 66.750952                71.414561   
                  >11                    71.328844                74.274025   
Year              2015                   65.946914                71.379642   
                  2016                   65.876810                71.621940   
                  2017                   66.520809                72.269282   
Time to Maturity  <= 1                   67.045159                71.939097   
                  (1-2]                  67.732875                72.594578   
                  (2-3]                  66.363575                72.275444   
                  (3-6]                  65.200706         

In [54]:
master.style.pipe(
    set_tex_style,
    caption=("master-long", "master-short"),
    label=f"{KEY}-master",
    bold_axis=0,
)


## Effective Spread 💴

In [55]:
results = []

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, X_print["mid"], out=eps_true, where=X_print["mid"] != 0)

rel_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")
nom_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")

for classifier in tqdm(classifiers):
    nom_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")
    rel_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")

    # calculate pred rel effective spread but not aggregated convert to %
    es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
    eps_pred = np.empty(es_pred.shape)
    np.divide(es_pred, X_print["mid"], out=eps_pred, where=X_print["mid"] != 0)

    # optional: quoted rel spread
    # (ask -bid) / ask

    wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

    res = pd.Series(
            {
                "nom_pred": nom_pred * 100,
                "rel_pred": rel_pred * 100,
                "nom_true": nom_true * 100,
                "rel_true": rel_true * 100,
                'statistic':wilcoxon_res.statistic,
                'pvalue':wilcoxon_res.pvalue
            }, name=classifier
        )
    results.append(res)

results = pd.concat(results, axis=1)

100%|██████████| 13/13 [00:53<00:00,  4.15s/it]


In [56]:
results.T.style.format("{:.3f}")

## Diffs 🔄️

In [57]:
# classical baselines -> for selection see "hyperparam-classical-ise_classical_supervised_val.tex"

base = master[
    [
        ("classical", "quote(best)->quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]

# my ml models
revised = master[
    [("gbm", "gbm(classical)"), ("gbm", "gbm(classical-size)"), ("gbm", "gbm(ml)")]
]


In [58]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:, i]).round(2)
        combo.sort_index(axis=1, inplace=True)

    return combo


In [59]:
diff = combine_results(revised, base)

diff.style.to_latex(
    f"../reports/diff-{KEY}.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:diff-{KEY}",
    caption=(f"long-diff-{KEY}", f"short-diff-{KEY}"),
    convert_css=True,
)


In [60]:
diff


gbm(classical)        gbm(classical-size)  \
                                          nom     pm                 nom   
Option Type       C                 65.512657   5.75           71.752108   
                  P                 66.606767   5.48           72.306912   
Security Type     Index option      60.309635   7.00           67.469911   
                  Others            69.039887   4.77           74.160293   
                  Stock option      65.318447   5.85           71.559532   
Trade Size        (0,1]             62.838447   6.18           70.814533   
                  (1,3]             64.907873   5.57           71.349116   
                  (3,5]             65.530185   5.44           72.022665   
                  (5,11]            66.584846   5.60           71.632683   
                  >11               71.162306   5.12           74.596742   
Year              2015              65.686244   4.85           71.517005   
                  2016              65.599281   5.75           71.689210   
                  2017              66.489807   5.60           72.401584   
Time to Maturity  <= 1              66.871548   5.36           72.041871   
                  (1-2]             67.586737   5.93           72.695265   
                  (2-3]             66.036792   6.04           72.431363   
                  (3-6]             65.029245   6.26           72.896311   
                  (6-12]            64.199075   5.91           71.966191   
                  > 12              56.015639   5.56           66.891540   
Moneyness         <= 0.7            65.757089   5.76           73.047039   
                  (0.7-0.9]         66.286506   5.85           72.245056   
                  (0.9-1.1]         66.989236   5.58           72.658136   
                  (1.1-1.3]         58.149819   5.14           66.258707   
                  > 1.3             56.912370   6.09           64.579940   
Location to Quote at mid            61.013630   6.74           67.961360   
                  inside            68.837832   3.59           72.024322   
                  at quotes         54.152059  16.00           74.856559   
                  outside           70.802418  -2.71           70.149766   
                  unknown           83.626840   0.94           83.392035   
All trades        all               66.010410   5.63           72.004510   

                                        gbm(ml)         
                                  pm        nom     pm  
Option Type       C             6.66  74.326804   9.23  
                  P             5.90  74.451890   8.04  
Security Type     Index option  1.73  72.346139   6.60  
                  Others        6.10  75.569654   7.51  
                  Stock option  6.85  74.093552   9.39  
Trade Size        (0,1]         7.56  73.435935  10.18  
                  (1,3]         7.16  73.647650   9.46  
                  (3,5]         6.57  74.190970   8.73  
                  (5,11]        6.16  74.157743   8.68  
                  >11           3.73  76.754620   5.89  
Year              2015          7.42  74.244782  10.14  
                  2016          7.34  74.197954   9.85  
                  2017          5.07  74.599385   7.27  
Time to Maturity  <= 1          6.06  74.239008   8.26  
                  (1-2]         6.10  75.288671   8.69  
                  (2-3]         6.02  74.814515   8.40  
                  (3-6]         7.06  75.145620   9.31  
                  (6-12]        6.93  74.428193   9.39  
                  > 12          7.67  71.214760  12.00  
Moneyness         <= 0.7        5.03  75.613996   7.59  
                  (0.7-0.9]     6.04  74.703549   8.50  
                  (0.9-1.1]     6.30  74.765376   8.41  
                  (1.1-1.3]     7.77  69.844057  11.35  
                  > 1.3         7.23  70.634098  13.28  
Location to Quote at mid        8.61  69.818239  10.46  
                  inside        6.71  73.714183   8.40  
  

## Statistics 📊

In [ ]:
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "ask_size_ex",
    "bid_size_ex",
    "TRADE_PRICE",
    "myn",
    "ROOT",
    "STRK_PRC",
    "OPTION_TYPE",
    "EXPIRATION",
]


train = pd.read_parquet(
        Path(data_dir, "train_set.parquet"), engine="fastparquet", columns=COLUMNS
)
val = pd.read_parquet(
        Path(data_dir, "val_set.parquet"), engine="fastparquet", columns=COLUMNS
)
test = pd.read_parquet(
        Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=COLUMNS
)
df = pd.concat([train, val, test])

In [ ]:
df["ttm"] = (df["EXPIRATION"] - df["QUOTE_DATETIME"]).dt.days
df["date"] = df["QUOTE_DATETIME"].dt.date

In [ ]:
# daily stats
stat_3 = df.groupby("date")[["ROOT"]].nunique().agg(["mean", "median", "std"])
stat_4 = df.groupby("date")[["ROOT"]].nunique().quantile([0.05, 0.5,  0.95])

df["key"] =df["OPTION_TYPE"].astype(str) + df["ROOT"].astype(str) + df["STRK_PRC"].astype(str) + df["EXPIRATION"].astype(str)

stat_7 = df.groupby("date")["key"].nunique().agg(["mean", "median", "std"])
stat_8 = df.groupby("date")["key"].nunique().quantile([0.05, 0.5,  0.95])

daily_stats = pd.concat([stat_3, stat_4], axis=0).T
daily_stats_2 = pd.concat([stat_7, stat_8], axis=0).T

In [ ]:
# non daily stats
rel_columns = ["TRADE_SIZE", "ttm", "myn"]
stat_1 = df[rel_columns].agg(["mean","median", "std"])
stat_2 = df[rel_columns].quantile([0.05, 0.5,  0.95])

other_stats = pd.concat([stat_1, stat_2], axis=0).T

In [ ]:
stats = pd.concat([daily_stats, daily_stats_2, other_stats], axis=0)

In [ ]:
stats